# ASHRAE ENERGY PREDICTION III:

# From Previous Notebook ....

## - Importing Necessary Libraries

In [1]:
# Importing needed libraries to be used throughout the project

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn import utils
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor,Ridge,ElasticNet
from sklearn.model_selection import train_test_split,KFold,GroupKFold
import lightgbm as lgb
import gc
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn. linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, Activation
from tensorflow.keras.optimizers import *

from prettytable import PrettyTable

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ashrae-energy-prediction/sample_submission.csv
/kaggle/input/ashrae-energy-prediction/building_metadata.csv
/kaggle/input/ashrae-energy-prediction/weather_train.csv
/kaggle/input/ashrae-energy-prediction/weather_test.csv
/kaggle/input/ashrae-energy-prediction/train.csv
/kaggle/input/ashrae-energy-prediction/test.csv


## - Loading and Reducing Memory Usage of Data

In [2]:
data_path = "/kaggle/input/ashrae-energy-prediction/"

train_path = data_path + "train.csv"

building_path = data_path + "building_metadata.csv"

weather_train_path = data_path + "weather_train.csv"

In [3]:
train_data = pd.read_csv(train_path)

building_data = pd.read_csv(building_path)

weather_train_data = pd.read_csv(weather_train_path)

In [4]:
# Converting data into feather format since some dataframes are too large and take a long time to load
# This method is inspired from the kaggle notebook titled: ASHRAE: feather format for fast loading
# Which is found at: https://www.kaggle.com/corochann/ashrae-feather-format-for-fast-loading

train_data.to_feather('train_data.feather')

building_data.to_feather('building_data.feather')

weather_train_data.to_feather('weather_train_data.feather')

In [5]:
train_data = pd.read_feather('train_data.feather')

building_data = pd.read_feather('building_data.feather')

weather_train_data = pd.read_feather('weather_train_data.feather')

In [6]:
# This function helps in optimizing the memory used by the dataframes by by modifying/altering thedatatype of each column.
# This method is inspired from the kaggle notebook titled: load data (reduce memory usage)
# Which is found at: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

def reduce_mem_usage(df, df_name):

    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:

        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2

    print('Memory usage of {} is reduced by {:.2f} %. Usage dropped from {:.2f} MB to {:.2f} MB.'.format(df_name, (100 * (start_mem - end_mem) / start_mem), start_mem, end_mem))
    
    return df


In [7]:
# Reducing Memory Usage of Data

train_data = reduce_mem_usage(train_data, 'Train Data')

building_data = reduce_mem_usage(building_data, 'Building Data')

weather_train_data = reduce_mem_usage(weather_train_data, 'Weather Train Data')

Memory usage of Train Data is reduced by 71.82 %. Usage dropped from 616.95 MB to 173.84 MB.
Memory usage of Building Data is reduced by 73.88 %. Usage dropped from 0.07 MB to 0.02 MB.
Memory usage of Weather Train Data is reduced by 73.06 %. Usage dropped from 9.60 MB to 2.59 MB.


In [8]:
# Merging Data

train = train_data.merge(building_data, on='building_id', how='left')
train = train.merge(weather_train_data, on=['site_id', 'timestamp'], how='left')

In [9]:
# Breaking Timestamp into Hour, Day, Month, Year

# This function firstly converts timestamp to date and then breaks down date into 6 new columns: hour, day, dayOfWeek, dayOfYear, month and year

def breakdown_timestamp(dataframe):
    
    dataframe['timestamp']= pd.to_datetime(dataframe['timestamp'])

    dataframe['hour']= np.uint8(dataframe['timestamp'].dt.hour)
    
    dataframe['day']= np.uint16(dataframe['timestamp'].dt.day)
    dataframe['dayofweek']= np.uint8(dataframe['timestamp'].dt.dayofweek)
    dataframe['dayofyear']= np.uint16(dataframe['timestamp'].dt.dayofyear)

    dataframe['month']= np.uint8(dataframe['timestamp'].dt.month)

    dataframe['year']= np.uint16(dataframe['timestamp'].dt.year)
    
    return dataframe

In [10]:
train = breakdown_timestamp(train)

## - Applying Log Transformation to 'Meter Reading and 'Square Feet'

In [11]:
train['meter_reading'] = np.log1p(train['meter_reading'])

In [12]:
train['square_feet'] = np.log1p(train['square_feet'])

## - Data Preperation and Feature Engineering

In [13]:
# Removing Zero Meter Readings

zero_meter_readings = list(train[train['meter_reading'] == 0].index)
train.drop(zero_meter_readings, axis = 0, inplace = True)

In [14]:
# Dropping Columns with More than 50 % Missing Values

threshold = len(train) * 0.5
train.dropna(axis=1, thresh = threshold, inplace = True)

In [15]:
# Filling Missing Values

train['cloud_coverage'].fillna(train['cloud_coverage'].median(), inplace=True)
train['sea_level_pressure'].fillna(train['sea_level_pressure'].median(), inplace=True)
train['precip_depth_1_hr'].fillna(train['precip_depth_1_hr'].median(), inplace=True)
train['wind_direction'].fillna(train['wind_direction'].median(), inplace=True)
train['wind_speed'].fillna(train['wind_speed'].median(), inplace=True)
train['dew_temperature'].fillna(train['dew_temperature'].median(), inplace=True)
train['air_temperature'].fillna(train['air_temperature'].median(), inplace=True)

In [16]:
# adding new feature from existing ones to get better results

train['season'] = train['timestamp'].apply(lambda x: 'Spring' if x.month==3 or x.month==4 or x.month==5 else 
                                                  'Summer' if x.month==6 or x.month==7 or x.month==8 else 
                                                  'Autumn' if x.month==9 or x.month==10 or x.month==11 else 
                                                  'Winter')

train['isDayTime'] = train['timestamp'].apply(lambda x: 1 if x.hour >=6 and x.hour <=18 else 0)

In [17]:
# Encoding categorical data

categorical_features = ['primary_use', 'season']

encoder = preprocessing.LabelEncoder()

for i in categorical_features:
    
    train[i] = encoder.fit_transform(train[i])
    

## - Dropping Timestamp and Reducing Memory Usage Again

In [18]:
train = train.drop(['timestamp'],axis=1)

In [19]:
reduced_train_data = reduce_mem_usage(train, 'Train Data')

Memory usage of Train Data is reduced by 36.62 %. Usage dropped from 1241.96 MB to 787.16 MB.


## - Removing Least Important Features Generated by our Feature Selection Method

In [20]:
new_data = reduced_train_data[['building_id','square_feet','primary_use','meter','site_id','air_temperature','dayofyear','hour','isDayTime','dew_temperature','dayofweek', 'meter_reading']]

In [21]:
new_data.drop(['site_id','dew_temperature'],axis=1, inplace = True)

# 8. Models and Fine Tuning

In [22]:
X_train = new_data.drop(['meter_reading'],axis = 1)

Y_train = new_data['meter_reading'].values

## 8.1 Basline Model

In [23]:
def baselineModel(y_actual,y_pred):

    rmsle_score = np.sqrt(np.mean((y_actual - y_pred) * (y_actual-y_pred)))
    
    print("The RMSLE Score of the Baseline Model is :",rmsle_score)

baselineModel(Y_train, np.median(Y_train))

The RMSLE Score of the Baseline Model is : 1.772


The baseline score can be computed is median value of labels. The baseline score for 50% of the data is 1.772. So it can be expected that the baseline score could be more.

**Splitting the data for training and testing**

In [24]:
train_x, test_x, train_y, test_y = train_test_split(X_train,Y_train, test_size=0.2, random_state=42)

**Calculating RMSLE**

RMSLE is the suggested evaluation metric for the models performance. Since we already applied log1p transformation to the target value 'Meter Reading', therefore, Root Mean Squared Error (RMSE) can be used directly. And can be computed as follows:

In [25]:
def RMSLE(y_actual, y_pred):
    
    return np.sqrt(mean_squared_error(y_actual, y_pred))

## 8.2 Linear Regression

In [26]:
linear_Regression = LinearRegression()
linear_Regression.fit(train_x, train_y)

LinearRegression()

**Calculating Prediction Score**

In [27]:
print('Linear Regression Traininig RMSLE = ', RMSLE((train_y) , (linear_Regression.predict(train_x))))
print('Linear Regression Testing RMSLE = ',RMSLE((test_y) ,(linear_Regression.predict(test_x))))

Linear Regression Traininig RMSLE =  1.3811028
Linear Regression Testing RMSLE =  1.3812215


**Linear Regresssion Cross Validation**

In [28]:
lin_scores = cross_val_score(linear_Regression, train_x, train_y,scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)

print("Linear Regression CV Scores:", lin_rmse_scores)
print("==========================================================================================")
print("Mean CV Score:", lin_rmse_scores.mean())

Linear Regression CV Scores: [1.38170799 1.38183736 1.38029682 1.37994614 1.38219218 1.38118286
 1.3830995  1.38021153 1.38149738 1.37906514]
Mean CV Score: 1.3811036903121514


## 8.3 ElasticNet

**Hyperparameter Tuning**

In [29]:
parameters = {'alpha':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'fit_intercept' : [False],
              'l1_ratio':[0.5]}

elasticnet = GridSearchCV(estimator = ElasticNet(),
                        param_grid = parameters,
                        cv = 3, 
                        scoring = 'neg_mean_squared_error',
                        verbose = 1,
                        return_train_score = True,
                        n_jobs = -1)

elasticnet.fit(train_x, train_y)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  1.8min finished


GridSearchCV(cv=3, estimator=ElasticNet(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'fit_intercept': [False], 'l1_ratio': [0.5]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=1)

In [30]:
print("Best ElasticNet Estimator is : ", elasticnet.best_estimator_)
print("Best ElasticNet Paramteres are : ", elasticnet.best_params_)

Best ElasticNet Estimator is :  ElasticNet(alpha=0.001, fit_intercept=False)
Best ElasticNet Paramteres are :  {'alpha': 0.001, 'fit_intercept': False, 'l1_ratio': 0.5}


**Building Best ElasticNet Estimator**

In [31]:
elastic_model = ElasticNet(alpha = 0.001, fit_intercept = False, l1_ratio = 0.5)
elastic_model.fit(train_x, train_y)

ElasticNet(alpha=0.001, fit_intercept=False)

**Calculating Prediction Score**

In [32]:
print('ElasticNet Training RMSLE = ',RMSLE((train_y) , (elastic_model.predict(train_x))))
print('ElasticNet Testing RMSLE = ',RMSLE((test_y) , (elastic_model.predict(test_x))))

ElasticNet Training RMSLE =  1.4735792
ElasticNet Testing RMSLE =  1.4732091


## 8.4 Ridge

**Hyperparameters Tuning**

In [33]:
parameters = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'fit_intercept' : [True],
              'solver' : ['lsqr']}

ridge = GridSearchCV(estimator = Ridge(),
                        param_grid = parameters,
                        cv = 3, 
                        scoring = 'neg_mean_squared_error',
                        verbose = 1,
                        return_train_score = True,
                        n_jobs = -1)

ridge.fit(train_x, train_y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   39.5s finished


GridSearchCV(cv=3, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'fit_intercept': [True], 'solver': ['lsqr']},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=1)

In [34]:
print("Best Ridge Estimator is :",ridge.best_estimator_)
print("Best Ridge Paramteres are : ", ridge.best_params_)

Best Ridge Estimator is : Ridge(alpha=0.0001, solver='lsqr')
Best Ridge Paramteres are :  {'alpha': 0.0001, 'fit_intercept': True, 'solver': 'lsqr'}


**Best Ridge Estimator**

In [35]:
Ridge = Ridge(alpha = 0.0001, fit_intercept = True, solver = "lsqr")
Ridge.fit(train_x, train_y)

Ridge(alpha=0.0001, solver='lsqr')

**Calculating Prediction Score**

In [36]:
print('Ridge Training RMSLE = ',RMSLE((train_y) , (Ridge.predict(train_x))))
print('Ridge Testing RMSLE = ',RMSLE(test_y , Ridge.predict(test_x)))

Ridge Training RMSLE =  1.3842031
Ridge Testing RMSLE =  1.384215


## 8.5 Lasso

**Hyperparameter Tuninig**

In [37]:
parameters = {'alpha':[0.001, 0.01, 0.1, 1, 10]}

lasso = GridSearchCV(estimator = Lasso(),
                        param_grid = parameters,
                        cv = 3, 
                        scoring = 'neg_mean_squared_error',
                        verbose = 1,
                        return_train_score = True,
                        n_jobs = -1)

lasso.fit(train_x, train_y)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   22.0s finished


GridSearchCV(cv=3, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=1)

In [38]:
print("Best Lasso Estimator ",lasso.best_estimator_)
print("Best Lasso Paramteres are : ", lasso.best_params_)

Best Lasso Estimator  Lasso(alpha=0.001)
Best Lasso Paramteres are :  {'alpha': 0.001}


**Best Lasso Estimator**

In [39]:
Lasso = Lasso(alpha = 0.001)
Lasso.fit(train_x, train_y)

Lasso(alpha=0.001)

**Calculating Prediction Score**

In [40]:
print('Lasso Traininig RMSLE = ',RMSLE((train_y) , (Lasso.predict(train_x))))
print('Lasso Testing RMSLE = ',RMSLE(test_y , Lasso.predict(test_x)))

Lasso Traininig RMSLE =  1.3811055
Lasso Testing RMSLE =  1.3812206


## 8.6 Decision Tree

**Hyperparameters Tuning**

In [41]:
parameters = {'max_depth': [3,5,7,9,11,15]}


decission_tree = GridSearchCV(estimator = DecisionTreeRegressor(),
                        param_grid = parameters,
                        cv = 3, 
                        scoring = 'neg_mean_squared_error',
                        verbose = 1,
                        return_train_score = True,
                        n_jobs = -1)

decission_tree.fit(train_x, train_y)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  5.9min finished


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7, 9, 11, 15]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=1)

In [42]:
print("Best Decision Tree Estimator ",decission_tree.best_estimator_)
print("Best Decision Tree Paramteres are : ", decission_tree.best_params_)

Best Decision Tree Estimator  DecisionTreeRegressor(max_depth=15)
Best Decision Tree Paramteres are :  {'max_depth': 15}


**Best Decision Tree Estimator**

In [43]:
DecissionTree = DecisionTreeRegressor(max_depth=15)
DecissionTree.fit(train_x, train_y)

DecisionTreeRegressor(max_depth=15)

**Calculating Prediction Score**

In [44]:
print('Decision Tree Training RMSLE = ',RMSLE((train_y) , (DecissionTree.predict(train_x))))
print('Decision Tree Testing RMSLE = ',RMSLE(test_y , DecissionTree.predict(test_x)))

Decision Tree Training RMSLE =  0.607702175679619
Decision Tree Testing RMSLE =  0.6099217534871779


## 8.7 RandomForestRegressor

**NOTE: Running this model will take so much time and may lead to crashing the session in process.**

**Hyperparameters Tuning**

In [45]:
parameters = { 'n_estimators': [60,80,100],
              'max_depth':[5,7,9]}

forest_reg = GridSearchCV(estimator = RandomForestRegressor(),
                        param_grid = parameters,
                        cv = 3, 
                        scoring = 'neg_mean_squared_error',
                        verbose = 1,
                        return_train_score = True,
                        n_jobs = -1)
forest_reg.fit(train_x, train_y)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
print("Best Random Forest Estimator ",forest_reg.best_estimator_)
print("Best Random Forest Paramteres are : ", forest_reg.best_params_)

**Best RandomForest Estimator**

In [ ]:
# RandomForest = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth)
# RandomForest.fit(train_x, train_y)

**Calculating Prediction Score**

In [ ]:
# print('Train RMSLE = ',RMSLE((train_y) , (RandomForest.predict(train_x))))
# print('Test RMSLE = ',RMSLE(test_y , RandomForest.predict(test_x)))

## 8.8 SGD Regressor

In [46]:
# Scaling the Data Prior to Passing it to the Model

x_train_scaled = preprocessing.scale(train_x)
x_test_scaled = preprocessing.scale(test_x)

**Hyperparameters Tuning**

In [47]:
parameters = {'alpha':[0.0001, 0.001],
             'eta0': [0.001],
             'penalty': ['l2'],
             'learning_rate': ['adaptive', 'invscaling'],
             'early_stopping': [True]}

sgd = GridSearchCV(estimator = SGDRegressor(),
                        param_grid = parameters,
                        cv = 3, 
                        scoring = 'neg_mean_squared_error',
                        verbose = 10,
                        return_train_score = True,
                        n_jobs = -1)

sgd.fit(x_train_scaled, train_y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  2.5min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  3.5min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  4.1min finished


GridSearchCV(cv=3, estimator=SGDRegressor(), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001], 'early_stopping': [True],
                         'eta0': [0.001],
                         'learning_rate': ['adaptive', 'invscaling'],
                         'penalty': ['l2']},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=10)

In [53]:
print("Best SGDRegressor Estimator ",sgd.best_estimator_)
print("Best SGDRegressor Paramteres are : ", sgd.best_params_)

Best SGDRegressor Estimator  SGDRegressor(early_stopping=True, eta0=0.001, learning_rate='adaptive')
Best SGDRegressor Paramteres are :  {'alpha': 0.0001, 'early_stopping': True, 'eta0': 0.001, 'learning_rate': 'adaptive', 'penalty': 'l2'}


**Best SGDRegressor**

In [54]:
sgd_regressor = SGDRegressor(early_stopping=True, eta0=0.001, learning_rate='adaptive')
sgd_regressor.fit(x_train_scaled, train_y)

SGDRegressor(early_stopping=True, eta0=0.001, learning_rate='adaptive')

**Calculating Prediction Score**

In [55]:
print('SGDRegressor Training RMSLE = ',RMSLE((train_y) , (sgd_regressor.predict(x_train_scaled))))
print('SGDRegressor Testing RMSLE = ',RMSLE(test_y , sgd_regressor.predict(x_test_scaled)))

SGDRegressor Training RMSLE =  1.3811032510904204
SGDRegressor Testing RMSLE =  1.381222083669289


## 8.9 SVR

**NOTE: Running this model will take so much time and may lead to crashing the session in process.**

**Building SVR Model**

In [ ]:
svm_reg = SVR(kernel="linear")
svm_reg.fit(x_train_scaled, train_y)

**Calculating Prediction Score**

In [ ]:
print('SVR Traininig RMSLE = ', RMSLE((train_y) , (svm_reg.predict(x_train_scaled))))
print('SVR Testing RMSLE = ',RMSLE((test_y) ,(svm_reg.predict(x_test_scaled))))

## 8.10 3 Layer Neural Network

In [57]:
from keras import backend as K

def NN_RMSLE(y_actual, y_pred):
    
    return K.sqrt(K.mean(K.square(y_pred - y_actual))) 

In [58]:
# Splitting Training Data into Trainig and Validation Sets

train_xx, val_xx, train_yy, val_yy = train_test_split(train_x,train_y, test_size=0.2, random_state=42)

In [59]:
from keras.callbacks import EarlyStopping

model = Sequential()

earlyStop= EarlyStopping(monitor='val_loss', mode='min', patience=3)

model.add(layers.Dense(512, activation='relu',input_shape=(train_xx.shape[1],)))
model.add(Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1  ,activation='linear'))

model.compile(optimizer='adam', loss = NN_RMSLE)

model.fit(train_xx, train_yy, epochs = 15, batch_size = 2048, validation_data=(val_xx,val_yy),callbacks = earlyStop)


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

Epoch 1/15
 549/5732 [=>............................] - ETA: 2:56 - loss: 2.4987[CV] alpha=0.0001, early_stopping=True, eta0=0.001, learning_rate=adaptive, penalty=l2 
[CV]  alpha=0.0001, early_stopping=True, eta0=0.001, learning_rate=adaptive, penalty=l2, score=(train=-1.907, test=-1.907), total= 2.0min
[CV] alpha=0.001, early_stopping=True, eta0=0.001, learning_rate=invscaling, penalty=l2 
[CV]  alpha=0.001, early_stopping=True, eta0=0.001, learning_rate=invscaling, penalty=l2, score=(train=-1.907, test=-1.907), total=  27.6s
[CV] alpha=0.001, early_stopping=True, eta0=0.001, learning_rate=invscaling, penalty=l2 
[CV]  alpha=0.001, early_stopping=True, eta0=0.001, learning_rate=invscaling, penalty=l2, score=(train=-1.907, test=-1.909), total=  27.7s
[CV] alpha=0.001, early_stopping=True, eta0=0.001, learning_rate=invscaling, penalty=l2 
[CV]  alpha=0.001, early_stopping=True, eta0=0.001, learning_rate=invscaling, penalty=l2, score=(train=-1.908, test=-1.906), total=  27.7s
 902/5732 

In [60]:
print('Neural Network Training RMSLE = ', model.evaluate(train_x, train_y, verbose=0))
print('Neural Network Testing RMSLE = ', model.evaluate(test_x, test_y, verbose=0))

Neural Network Training RMSLE =  2.4793498516082764
Neural Network Testing RMSLE =  2.4787943363189697


## 8.11 Scores Comparison

In [61]:
scores_table = PrettyTable(["Model Name","Training RMSLE","Testing RMSLE"])

scores_table.add_row(["LinearRegression", "1.3811028", "1.3812215"])
scores_table.add_row(["ElasticNet", "1.4735792", "1.4732091"])
scores_table.add_row(["Ridge", "1.3842031", "1.3842150"])
scores_table.add_row(["Lasso", "1.3811055", "1.3812206"])
scores_table.add_row(["DecisionTree", "0.6077021", "0.6098925"])
scores_table.add_row(["RandomForestRegressor", "--------", "--------"])
scores_table.add_row(["SGDRegressor", "1.3811040", "1.3812219"])
scores_table.add_row(["SVR", "--------", "--------"])
scores_table.add_row(["Neural Network", "2.2980730", "2.2978658"])

print(scores_table)

+-----------------------+----------------+---------------+
|       Model Name      | Training RMSLE | Testing RMSLE |
+-----------------------+----------------+---------------+
|    LinearRegression   |   1.3811028    |   1.3812215   |
|       ElasticNet      |   1.4735792    |   1.4732091   |
|         Ridge         |   1.3842031    |   1.3842150   |
|         Lasso         |   1.3811055    |   1.3812206   |
|      DecisionTree     |   0.6077021    |   0.6098925   |
| RandomForestRegressor |    --------    |    --------   |
|      SGDRegressor     |   1.3811040    |   1.3812219   |
|          SVR          |    --------    |    --------   |
|     Neural Network    |   2.2980730    |   2.2978658   |
+-----------------------+----------------+---------------+


**observations:**
1. Out of all tested models, Decision Tree Regressor performed the best with an approximate RMSLE score of 0.6.
2. Linear Regression, Ridge, Lasso and SGDRegressor had compitetively close scores, with Linear Regression talking the lead as the second best prediction method.
3. Our constructed Neural Network Performed worst out of all tested models, indicating its unsuitability for such problem.

# 9.  Analyzing the Best Model

##  9.1 Viewing the Scores of the Hyperparameter Combinations Tested During the Grid Search

In [62]:
pd.DataFrame(decission_tree.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,30.029331,0.316666,0.538437,0.005045,3,{'max_depth': 3},-1.914567,-1.916316,-1.912353,-1.914412,0.001622,6,-1.914330,-1.913455,-1.915438,-1.914408,0.000811
1,48.106115,0.292071,0.629844,0.009377,5,{'max_depth': 5},-1.605538,-1.606929,-1.602004,-1.604824,0.002073,5,-1.604449,-1.603746,-1.606211,-1.604802,0.001037
2,64.928814,0.124299,0.720273,0.003717,7,{'max_depth': 7},-1.296241,-1.297142,-1.293259,-1.295547,0.001660,4,-1.295030,-1.294548,-1.296523,-1.295367,0.000841
3,80.793695,0.706668,0.883922,0.019023,9,{'max_depth': 9},-1.019235,-1.015168,-1.012483,-1.015629,0.002776,3,-1.016904,-1.013273,-1.015566,-1.015248,0.001499
4,94.326274,0.335534,1.040894,0.013341,11,{'max_depth': 11},-0.722919,-0.718741,-0.719851,-0.720504,0.001767,2,-0.721179,-0.716033,-0.721682,-0.719631,0.002553
5,97.701526,9.707439,1.356205,0.185781,15,{'max_depth': 15},-0.374326,-0.370895,-0.368215,-0.371145,0.002501,1,-0.369542,-0.364535,-0.364631,-0.366236,0.002338


## 9.2 Viewing the Relative Importance of Each Attribute for Making Accurate Predictions

In [63]:
features_list = ['building_id','square_feet','primary_use','meter','air_temperature','dayofyear','hour','isDayTime','dayofweek']

features_importance = decission_tree.best_estimator_.feature_importances_

sorted(zip(features_importance, features_list), reverse=True)

[(0.5054091863350517, 'square_feet'),
 (0.2535869754308519, 'building_id'),
 (0.10493044669541486, 'meter'),
 (0.051062695128919984, 'primary_use'),
 (0.050448430477704885, 'air_temperature'),
 (0.01838587813098989, 'dayofyear'),
 (0.007122934006580058, 'isDayTime'),
 (0.006197295542875907, 'hour'),
 (0.002856158251610588, 'dayofweek')]

**Observations**
1. Square Feet attribute contributes the most towards predicting the amount of energy consumption.
2. Building Id is the second highly influencing factor, with an importance of 0.25.
3. It seems that attributes relating to time and date do not affect energy consumption much.

**Ensemble Methods**
- We believe that Random Forest Regressor might offer better performance as they rely on Decision Trees and this ensemble can further enhance performance. Unfortunately, due to extremely slow running times/sessions crashing, we were unable to test this theory.